<div style="display: flex; justify-content: space-between; align-items: center;">
    <div style="text-align: left; flex: 4;">
        <strong>Author:</strong> Amirhossein Heydari — 
        📧 <a href="mailto:amirhosseinheydari78@gmail.com">amirhosseinheydari78@gmail.com</a> — 
        🐙 <a href="https://github.com/mr-pylin/media-processing-workshop" target="_blank" rel="noopener">github.com/mr-pylin</a>
    </div>
    <div style="display: flex; justify-content: flex-end; flex: 1; gap: 8px; align-items: center; padding: 0;">
        <a href="https://opencv.org/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/opencv/logo/OpenCV_logo_no_text-1.svg"
                 alt="OpenCV Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://pillow.readthedocs.io/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/pillow/logo/pillow-logo-248x250.png"
                 alt="PIL Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://scikit-image.org/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/scikit-image/logo/logo.png"
                 alt="scikit-image Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://scipy.org/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/scipy/logo/logo.svg"
                 alt="SciPy Logo"
                 style="max-height: 48px; width: auto;">
        </a>
    </div>
</div>
<hr>


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Load an Image](#toc2_)    
  - [Image Degradation](#toc2_1_)    
- [Quality Assessment](#toc3_)    
  - [Full-Reference (FR-IQA) Metrics](#toc3_1_)    
    - [Pixel-Based Metrics](#toc3_1_1_)    
      - [Mean Squared Error (MSE)](#toc3_1_1_1_)    
        - [Manual](#toc3_1_1_1_1_)    
        - [Using scikit-image](#toc3_1_1_1_2_)    
      - [Peak Signal-to-Noise Ratio (PSNR)](#toc3_1_1_2_)    
        - [Manual](#toc3_1_1_2_1_)    
        - [Using scikit-image](#toc3_1_1_2_2_)    
    - [Structural and Perceptual Metrics](#toc3_1_2_)    
      - [Structural Similarity Index (SSIM)](#toc3_1_2_1_)    
        - [Using scikit-image](#toc3_1_2_1_1_)    
      - [Multi-Scale Structural Similarity (MS-SSIM)](#toc3_1_2_2_)    
        - [Manual](#toc3_1_2_2_1_)    
      - [Feature Similarity Index (FSIM)](#toc3_1_2_3_)    
      - [Visual Information Fidelity (VIF)](#toc3_1_2_4_)    
  - [Reduced-Reference (RR-IQA) Metrics](#toc3_2_)    
    - [Reduced-Reference Entropy Difference (RRED)](#toc3_2_1_)    
    - [Wavelet-Based Reduced Reference (WBRR)](#toc3_2_2_)    
    - [Reduced-Reference Image Quality Assessment (RRIQA)](#toc3_2_3_)    
  - [No-Reference (NR-IQA) Metrics](#toc3_3_)    
    - [Blind/Referenceless Image Spatial Quality Evaluator (BRISQUE)](#toc3_3_1_)    
    - [Natural Image Quality Evaluator (NIQE)](#toc3_3_2_)    
    - [Perception-based Image Quality Evaluator (PIQE)](#toc3_3_3_)    
    - [No-Reference Image Quality Assessment via Transformers (NR-IQA)](#toc3_3_4_)    
    - [MetaIQA](#toc3_3_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage
from numpy.typing import NDArray

In [ ]:
# disable automatic figure display (plt.show() required)  
# this ensures consistency with .py scripts and gives full control over when plots appear
plt.ioff()

# <a id='toc2_'></a>[Load an Image](#toc0_)

In [ ]:
im_1 = cv2.imread("../../assets/images/dip_3rd/CH02_Fig0222(b)(cameraman).tif", flags=cv2.IMREAD_GRAYSCALE)

In [ ]:
# plot
plt.imshow(im_1, cmap="gray")
plt.title("CH02_Fig0222(b)(cameraman).tif")
plt.axis("off")
plt.show()

## <a id='toc2_1_'></a>[Image Degradation](#toc0_)

📝 **Docs**:

- `cv2.filter2D`: [docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga27c049795ce870216ddfb366086b5a04](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga27c049795ce870216ddfb366086b5a04)


In [ ]:
# negative
max_value = 2 ** np.iinfo(im_1.dtype).bits - 1
im_1_negative = max_value - im_1

In [ ]:
# gaussian noise
gaussian_noise = np.random.default_rng(seed=42).normal(loc=0, scale=25, size=im_1.shape)
im_1_gaussian_noise = np.clip(im_1.astype(np.float64) + gaussian_noise, 0, 255).astype(np.uint8)

In [ ]:
# periodic noise (sinusoidal)
amplitude = 50
frequency = 10
rows, cols = im_1.shape
X, Y = np.meshgrid(np.arange(cols), np.arange(rows))
periodic_noise = amplitude * np.sin(2 * np.pi * frequency * X / cols + 2 * np.pi * frequency * Y / rows)
im_1_periodic_noise = np.clip(im_1.astype(np.float64) + periodic_noise, 0, 255).astype(np.uint8)

In [ ]:
# motion blur
size = 5
kernel = np.eye(size) / size
im_1_blur = cv2.filter2D(im_1, -1, kernel)

In [ ]:
# plot
titles = ["Negative", "Gaussian Noise", "Periodic Noise", "Motion Blur"]
images = [im_1_negative, im_1_gaussian_noise, im_1_periodic_noise, im_1_blur]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 4), layout="compressed")
for i, ax in enumerate(fig.axes):
    ax.imshow(images[i], cmap="gray", vmin=0, vmax=255)
    ax.set_title(titles[i])
plt.show()

# <a id='toc3_'></a>[Quality Assessment](#toc0_)

- It's used to measure the **degradation** caused by various operations like **filtering**, **compression**, **noise**, and **enhancement**.

📝 **Docs**:

- `skimage.metrics`: [scikit-image.org/docs/stable/api/skimage.metrics.html](https://scikit-image.org/docs/stable/api/skimage.metrics.html)


## <a id='toc3_1_'></a>[Full-Reference (FR-IQA) Metrics](#toc0_)

- These methods compare a **processed** image against a high-quality original (**reference**) image.

<table style="margin:0 auto;">
  <tr>
    <th>Metric</th>
    <th>Range</th>
    <th>Better Quality</th>
  </tr>
  <tr>
    <td>Mean Squared Error (MSE)</td>
    <td>[0, ∞)</td>
    <td>Lower</td>
  </tr>
  <tr>
    <td>Peak Signal-to-Noise Ratio (PSNR)</td>
    <td>[0, ∞), typically 30-50 dB</td>
    <td>Higher</td>
  </tr>
  <tr>
    <td>Structural Similarity Index (SSIM)</td>
    <td>[-1, 1], typically [0,1]</td>
    <td>Higher</td>
  </tr>
  <tr>
    <td>Multi-Scale Structural Similarity (MS-SSIM)</td>
    <td>[0, 1]</td>
    <td>Higher</td>
  </tr>
  <tr>
    <td>Feature Similarity Index (FSIM)</td>
    <td>[0, 1]</td>
    <td>Higher</td>
  </tr>
  <tr>
    <td>Visual Information Fidelity (VIF)</td>
    <td>[0, ∞), typically [0,1]</td>
    <td>Higher</td>
  </tr>
</table>


### <a id='toc3_1_1_'></a>[Pixel-Based Metrics](#toc0_)

- Pixel-based metrics directly compare the pixel values of the reference and test images.
- They are straightforward and computationally efficient but often do not correlate well with human perception of image quality.


#### <a id='toc3_1_1_1_'></a>[Mean Squared Error (MSE)](#toc0_)

- Measures the average squared difference between the original and distorted images.

$$\text{MSE} = \frac{1}{MN} \sum_{i=1}^{M} \sum_{j=1}^{N} \left( I(i,j) - K(i,j) \right)^2$$


##### <a id='toc3_1_1_1_1_'></a>[Manual](#toc0_)


In [ ]:
def mse(image_1: NDArray, image_2: NDArray) -> float:
    return ((image_1.astype(np.float64) - image_2.astype(np.float64)) ** 2).mean().item()

In [ ]:
print(f"mean-squared-error")
for i in range(len(images)):
    print(f"\t{titles[i]:{len(max(titles))}} : {mse(im_1, images[i])}")

##### <a id='toc3_1_1_1_2_'></a>[Using scikit-image](#toc0_)


In [ ]:
print(f"mean-squared-error")
for i in range(len(images)):
    print(f"\t{titles[i]:{len(max(titles))}} : {skimage.metrics.mean_squared_error(im_1, images[i]).item()}")

#### <a id='toc3_1_1_2_'></a>[Peak Signal-to-Noise Ratio (PSNR)](#toc0_)

- Derived from MSE, it indicates the ratio of the maximum possible power of a signal to the power of corrupting noise.

$$\text{PSNR} = 10 \log_{10} \left( \frac{\text{MAX}^2}{\text{MSE}} \right)$$


##### <a id='toc3_1_1_2_1_'></a>[Manual](#toc0_)


In [ ]:
def psnr(image_1: NDArray, image_2: NDArray) -> float:
    max_value = 2 ** (np.iinfo(image_1.dtype).bits) - 1
    mse_value = np.mean((image_1.astype(np.float64) - image_2.astype(np.float64)) ** 2)
    if mse_value == 0:
        return float("inf")
    return 10 * np.log10((max_value**2) / mse_value)

In [ ]:
print(f"peak-signal-noise-ratio")
for i in range(len(images)):
    print(f"\t{titles[i]:{len(max(titles))}} : {psnr(im_1, images[i])}")

##### <a id='toc3_1_1_2_2_'></a>[Using scikit-image](#toc0_)

In [ ]:
print(f"peak-signal-noise-ratio")
for i in range(len(images)):
    print(f"\t{titles[i]:{len(max(titles))}} : {skimage.metrics.peak_signal_noise_ratio(im_1, images[i])}")

### <a id='toc3_1_2_'></a>[Structural and Perceptual Metrics](#toc0_)

- Perceptual metrics aim to model the human visual system (HVS) and assess image quality based on how humans perceive differences between images.
-  These metrics consider factors like luminance, contrast, and structural information.


#### <a id='toc3_1_2_1_'></a>[Structural Similarity Index (SSIM)](#toc0_)

- Evaluates image quality based on changes in structural information, luminance, and contrast.

📝 **Paper**:

- [Image quality assessment: from error visibility to structural similarity](https://ece.uwaterloo.ca/~z70wang/publications/ssim.pdf)


##### <a id='toc3_1_2_1_1_'></a>[Using scikit-image](#toc0_)


In [ ]:
print(f"structural-similarity")
for i in range(len(images)):
    print(
        f"\t{titles[i]:{len(max(titles))}} : {skimage.metrics.structural_similarity(im_1, images[i], win_size=11, sigma=1.5, gaussian_weights=True, K1=0.01, K2=0.03)}"
    )

#### <a id='toc3_1_2_2_'></a>[Multi-Scale Structural Similarity (MS-SSIM)](#toc0_)

- It is an extension of the Structural Similarity Index (SSIM) that evaluates image quality across multiple spatial scales.

📝 **Paper**:

- [Multiscale structural similarity for image quality assessment](https://utw10503.utweb.utexas.edu/publications/2003/zw_asil2003_msssim.pdf)


##### <a id='toc3_1_2_2_1_'></a>[Manual](#toc0_)


In [ ]:
def gaussian_filter(size=11, sigma=1.5):
    x = np.linspace(-size // 2, size // 2, size)
    gauss = np.exp(-(x**2) / (2 * sigma**2))
    gauss = gauss / gauss.sum()
    kernel = np.outer(gauss, gauss)
    return kernel


def downsample(image):
    image = cv2.filter2D(image, -1, gaussian_filter())
    return cv2.pyrDown(image)


def ms_ssim(img1, img2, levels=5, weight_factors=None):
    if weight_factors is None:
        weight_factors = [0.0448, 0.2856, 0.3001, 0.2363, 0.1333]  # default weights from the paper

    msssim = []
    epsilon = 1e-8  # small value to prevent numerical instability

    for _ in range(levels):
        data_range = max(img1.max() - img1.min(), epsilon)  # avoid zero data range
        score, _ = skimage.metrics.structural_similarity(img1, img2, full=True, data_range=data_range)

        score = np.fabs(score)  # use absolute value instead of clamping
        msssim.append(score)

        img1, img2 = downsample(img1), downsample(img2)

    return np.prod(np.array(msssim) ** np.array(weight_factors))

In [ ]:
print(f"multi-scale-structural-similarity")
for i in range(len(images)):
    print(f"\t{titles[i]:{len(max(titles))}} : {ms_ssim(im_1, images[i])}")

#### <a id='toc3_1_2_3_'></a>[Feature Similarity Index (FSIM)](#toc0_)

- Uses phase congruency and gradient magnitude to assess image quality.

📝 **Paper**:

- [FSIM: A Feature Similarity Index for Image Quality Assessment](https://ieeexplore.ieee.org/abstract/document/5705575)


#### <a id='toc3_1_2_4_'></a>[Visual Information Fidelity (VIF)](#toc0_)

- Measures the amount of information that can be extracted by the human visual system from the distorted image relative to the reference image.

📝 **Paper**:

- [Image information and visual quality](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=58d0d3b905c6531e25d94b8c20605a0e1e1ba1cb)


## <a id='toc3_2_'></a>[Reduced-Reference (RR-IQA) Metrics](#toc0_)

- These methods require only **partial** information from the **reference** image.

<table style="margin:0 auto;">
  <tr>
    <th>Metric</th>
    <th>Range</th>
    <th>Better Quality</th>
  </tr>
  <tr>
    <td>Reduced-Reference Entropy Difference (RRED)</td>
    <td>[0, ∞)</td>
    <td>Lower</td>
  </tr>
  <tr>
    <td>Wavelet-Based Reduced Reference (WBRR)</td>
    <td>[0, 1]</td>
    <td>Higher</td>
  </tr>
  <tr>
    <td>Reduced-Reference Image Quality Assessment (RRIQA)</td>
    <td>[0, 1]</td>
    <td>Higher</td>
  </tr>
</table>


### <a id='toc3_2_1_'></a>[Reduced-Reference Entropy Difference (RRED)](#toc0_)

- Uses entropy differences between the reference and distorted images.

📝 **Paper**:

- [RRED Indices: Reduced Reference Entropic Differencing for Image Quality Assessment](https://ieeexplore.ieee.org/document/5999718)


### <a id='toc3_2_2_'></a>[Wavelet-Based Reduced Reference (WBRR)](#toc0_)

- Utilizes wavelet transform coefficients to compare the reference and distorted images.


### <a id='toc3_2_3_'></a>[Reduced-Reference Image Quality Assessment (RRIQA)](#toc0_)

- Employs natural scene statistics in the wavelet domain to evaluate image quality.


## <a id='toc3_3_'></a>[No-Reference (NR-IQA) Metrics](#toc0_)

- These methods work **without** a **reference** image.

<table style="margin:0 auto;">
  <tr>
    <th>Metric</th>
    <th>Range</th>
    <th>Better Quality</th>
  </tr>
  <tr>
    <td>BRISQUE (Blind/Referenceless Image Spatial Quality Evaluator)</td>
    <td>[0, 100]</td>
    <td>Lower</td>
  </tr>
  <tr>
    <td>NIQE (Natural Image Quality Evaluator)</td>
    <td>[0, ∞), typically [0, 10]</td>
    <td>Lower</td>
  </tr>
  <tr>
    <td>PIQE (Perception-based Image Quality Evaluator)</td>
    <td>[0, 100]</td>
    <td>Lower</td>
  </tr>
  <tr>
    <td>No-Reference Image Quality Assessment via Transformers (NR-IQA)</td>
    <td>Depends on implementation</td>
    <td>Varies</td>
  </tr>
  <tr>
    <td>MetaIQA</td>
    <td>Depends on implementation</td>
    <td>Varies</td>
  </tr>
</table>


### <a id='toc3_3_1_'></a>[Blind/Referenceless Image Spatial Quality Evaluator (BRISQUE)](#toc0_)

- Uses natural scene statistics to evaluate image quality (**Requires Pretrained Model**).

**Pretrained Models**:

- `brisque_model_live.yml`: [github.com/opencv/opencv_contrib/blob/master/modules/quality/samples/brisque_model_live.yml](https://github.com/opencv/opencv_contrib/blob/master/modules/quality/samples/brisque_model_live.yml)
- `brisque_range_live.yml`: [github.com/opencv/opencv_contrib/blob/master/modules/quality/samples/brisque_range_live.yml](https://github.com/opencv/opencv_contrib/blob/master/modules/quality/samples/brisque_range_live.yml)


### <a id='toc3_3_2_'></a>[Natural Image Quality Evaluator (NIQE)](#toc0_)

- Measures deviations from statistical regularities observed in natural images.


### <a id='toc3_3_3_'></a>[Perception-based Image Quality Evaluator (PIQE)](#toc0_)

- Computes a perceptual quality score based on block-wise distortion in the image.


### <a id='toc3_3_4_'></a>[No-Reference Image Quality Assessment via Transformers (NR-IQA)](#toc0_)

- Leverages deep learning models like CNNs and Transformers to predict image quality.


### <a id='toc3_3_5_'></a>[MetaIQA](#toc0_)

- Uses deep meta-learning to adapt to various distortions and assess image quality.
